The purpose of this code is to search GTDB-Tk reference genome for gyrase B sequences to be able to assign species-level taxonomy to gyraseB amplicon data. Genomes are annotated using Prodigal, then gyraseB sequences are filtered out using a HMM profile of gyrase B from the TIGR01059 profile in the individual hmms folder. Then nucleotide and protein sequences are output to fastas. These fastas will be used as reference databases for local blast searches used to assign taxonomy. 

Inputs: genome fastas, TIGR01059 gyrb HMM profile

Outputs: blast database of full length gyrb seq used to assign taxonomy and
additional fastas of just Bacteroidales seqs or Bacteroidales seqs plus outgroups 

In [5]:
import os
import pandas as pd
from Bio import SeqIO
from Bio import Seq

os.getcwd()
os.chdir('/Volumes/AHN/captive_ape_microbiome')


#define input/output folders
indir =  'data/gyrb/ref_gyrb_gtdbtk/GTDBTK_db'
outdir = 'results/gyrb/processing/ref_gyrb_gtdbtk'

os.system('mkdir -pv '+outdir+'/hmm_out')
os.system('mkdir -pv '+outdir+'/prodigal')
os.system('mkdir -pv '+outdir+'/gyrb_seqs')
os.system('mkdir -pv '+outdir+'/blastdb')
os.system('mkdir -pv '+outdir+'/gyrb_fastas')

0

In [32]:
#Read in taxonomy file
tax = pd.read_csv(f'{indir}/taxonomy/gtdb_taxonomy.tsv',sep='\t',header=None) 
tax.columns = ["gtdbtk_genome", "taxonomy"]
tax['ncbi_genome'] = tax['gtdbtk_genome'].str.split('_',n=1,expand=True).loc[:,1]
gtdbtk_to_ncbi = tax.set_index('gtdbtk_genome')['ncbi_genome'].to_dict()
ncbi_to_tax = tax.set_index('ncbi_genome')['taxonomy'].to_dict()
tax[['Domain','Phylum','Class','Order','Family','Genus','Species']] = tax.taxonomy.apply( 
   lambda x: pd.Series(str(x).split(";"))) 


In [52]:
#Select only Bacteroidales genomes
Bt = tax[tax['taxonomy'].str.contains('Bacteroidales')]
print(len(Bt),'Bacteroidales genomes in GTDBTK database')

#Select representatives, one from each other Order
not_Bt = tax[~tax['taxonomy'].str.contains('Bacteroidales')]
not_Bt_reps = not_Bt.groupby('Order').head(1)
print(len(not_Bt_reps),'Other bacterial orders in GTDBTK database')

#write out taxonomy file
Bt_plusreps = pd.concat([Bt, not_Bt_reps])
Bt_plusreps.to_csv(outdir+'/Bt_plusreps.txt',index=False,sep='\t')
print(len(Bt_plusreps),'Total genomes in GTDBTK database added to assign taxonomy')

1029 Bacteroidales genomes in GTDBTK database
924 Other bacterial orders in GTDBTK database
1953 Total genomes in GTDBTK database added to assign taxonomy


### Annotate GTDB-Tk Bacteroidales and other taxonomic representative genomes

In [45]:
def prodigal(genome):
    #Annotate genomes using prodigal and identifies gyrb gene using a gtdbtk hmmprofile 
    os.system(f'gunzip {indir}/fastani/database/{genome}_genomic.fna.gz')
    os.system(f'prodigal -i {indir}/fastani/database/{genome}_genomic.fna -d {outdir}/prodigal/{genome}_genomic.fna -a {outdir}/prodigal/{genome}_genomic.faa')
    os.system(f'hmmsearch --tblout {outdir}/hmm_out/{genome}.txt {indir}/markers/tigrfam/individual_hmms/TIGR01059.HMM {outdir}/prodigal/{genome}_genomic.faa')
    os.system(f'gzip {indir}/fastani/database/{genome}_genomic.fna')   
    
prodigal('GCF_002849695.1')

In [49]:
#list genomes with fastas
genome_files = [f.split('_genomic.fna.gz')[0] for f in os.listdir(f'{indir}/fastani/database/')]

#select only those genomes in Bt_plusreps
Bt_plusreps = Bt_plusreps[Bt_plusreps['ncbi_genome'].isin(genome_files)]
print(len(Bt_plusreps),'genomes') #there are several reps listed in metadata without genomes in the GTDBTK folder

#how many have already been annotated
annotated_genomes = [f.split('_genomic')[0] for f in os.listdir(f'{outdir}/prodigal') if f.endswith('.faa')]
# test run
#annotated_genomes = annotated_genomes[:-5]
print(len(annotated_genomes),'genomes already annotated')

genomes_to_do = set(Bt_plusreps['ncbi_genome'])-set(annotated_genomes) 
print(len(genomes_to_do),'genomes to annotate') 

1945 genomes
1945 genomes already annotated
0 genomes to annotate
set()


In [7]:
#Annotate genomes that haven't been annotated
for genome in genomes_to_do:
    prodigal(genome,outdir)             

### Identify best gyrb hit and output fasta

In [8]:
def get_besthit(genome):
    #parse hmmsearch result to return sequence header and evalue
    try:
        df = pd.read_csv(f'{outdir}/hmm_out/{genome}.txt',delim_whitespace=True,header=None,comment='#')
        besthit = df.iloc[0]
        name=besthit[0]
        evalue=besthit[4]
        return(pd.Series([name,evalue]))
    except:
        return(pd.Series(['no_hit','NA']))
#test 
print(get_besthit('GCF_002849695.1'))
   
def write_fasta(genome,besthit):
    record_dict = SeqIO.to_dict(SeqIO.parse(f"{outdir}/prodigal/{genome}_genomic.fna", "fasta"))
    besthit_record = record_dict[besthit]
    besthit_record.id = genome
    besthit_record.description=ncbi_to_tax[genome]
    if len(besthit_record.seq) < 1800:
        return('partial_len')
    elif 'N' in str(besthit_record.seq): #ensures gene is fulllength
        return('ambig_char')
    else:
        SeqIO.write(besthit_record, f"{outdir}/gyrb_seqs/{genome}.fasta", "fasta")
        return('passing_hit')
#test
write_fasta('GCF_002849695.1','NZ_CP018937.1_3992')  

Bt_plusreps[['best_hit','evalue']] = Bt_plusreps['ncbi_genome'].apply(get_besthit)
#filter out seqs with no hit
Bt_plusreps_hits = Bt_plusreps[Bt_plusreps['best_hit']!='no_hit']
print(len(Bt_plusreps_hits),'gyrB seqs found')
Bt_plusreps_hits = Bt_plusreps_hits[Bt_plusreps_hits['evalue'].astype(float)<float(1e-250)]
print(len(Bt_plusreps_hits),'gyrB seqs passing eval threshold')

#remove fastas
os.system(f"rm -r {outdir}/gyrb_seqs/")
os.system(f"mkdir -pv {outdir}/gyrb_seqs/")

Bt_plusreps_hits['records'] = Bt_plusreps_hits.apply(
                                lambda row: write_fasta(row['ncbi_genome'],row['best_hit']),
                                axis = 1)
print('summary of gyrb hits')
print(Bt_plusreps_hits['records'].value_counts())
print('summary of Bt gyrb hits')
print(Bt_plusreps_hits['records'][Bt_plusreps_hits['Order']=='o__Bacteroidales'].value_counts())

Bt_plusreps_hits.to_csv(f'{outdir}/Bt_plusreps_hits.txt',sep='\t',index=False)

0    NZ_CP018937.1_3992
1              3.5e-287
dtype: object
1859 gyrB seqs found
1621 gyrB seqs passing eval threshold
summary of gyrb hits
passing_hit    1561
ambig_char       50
partial_len      10
Name: records, dtype: int64
summary of Bt gyrb hits
passing_hit    890
ambig_char       7
partial_len      3
Name: records, dtype: int64


### concat gyrb fastas, translate, make blastdbs for assigning taxonomy

In [3]:
%%bash
cd results/gyrb/processing/ref_gyrb_gtdbtk

#concatenate gyrb seqs to file
cat gyrb_seqs/*.fasta > blastdb/gtdbtk_gyrb.fasta

#translate, align aa, and uses a guide for nucleotide seqs
transeq -sequence blastdb/gtdbtk_gyrb.fasta -outseq  blastdb/gtdbtk_gyrb.faa

makeblastdb -in blastdb/gtdbtk_gyrb.fasta -dbtype nucl
makeblastdb -in blastdb/gtdbtk_gyrb.faa -dbtype prot



Building a new DB, current time: 06/26/2020 14:44:18
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb/processing/ref_gyrb_gtdbtk/blastdb/gtdbtk_gyrb.fasta
New DB title:  blastdb/gtdbtk_gyrb.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Volumes/AHN/captive_ape_microbiome/results/gyrb/processing/ref_gyrb_gtdbtk/blastdb/gtdbtk_gyrb.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1561 sequences in 0.0744228 seconds.


Building a new DB, current time: 06/26/2020 14:44:18
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb/processing/ref_gyrb_gtdbtk/blastdb/gtdbtk_gyrb.faa
New DB title:  blastdb/gtdbtk_gyrb.faa
Sequence type: Protein
Deleted existing Protein BLAST database named /Volumes/AHN/captive_ape_microbiome/results/gyrb/processing/ref_gyrb_gtdbtk/blastdb/gtdbtk_gyrb.faa
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1561 sequences in 0.051151 seconds.

Translate nucleic acid sequences


### generate additional Bacteroidales-only fastas 

In [11]:
#select only Bacteroidales seqs and realign

with open(f'{outdir}/blastdb/gtdbtk_gyrb.fasta') as original: 
    with open(f'{outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.fasta', 'w') as Bt_only:
        records = SeqIO.parse(original, 'fasta')
        for record in records:
            if 'o__Bacteroidales' in record.description or 'p__Gemmatimonadota' in record.description or 'c__Chlorobia' in record.description or 'c__Ignavibacteria' in record.description:
                description = record.description.split(';')
                phylum = description[1]
                order = description[3]
                family = description[4]
                genus_sp = description[6].replace(' ','_')
                record.id = record.id + phylum + order + family + genus_sp
                record.description = ''
                if 'o__Bacteroidales' not in record.id:
                    print(record.id)
                SeqIO.write(record, Bt_only, 'fasta')

os.system(f'transeq -sequence {outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.fasta -outseq {outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.faa')
os.system(f'mafft --auto --quiet {outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.faa > {outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.faa.aln')
os.system(f'tranalign -asequence  {outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.fasta -bsequence {outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.faa.aln -outseq {outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.fasta.aln')


GCA_001780825.1p__Gemmatimonadotao__GWA2-58-10f__GWA2-58-10s__GWA2-58-10_sp001780825
GCA_002348465.1p__Gemmatimonadotao__SG8-23f__UBA6960s__UBA2589_sp002348465
GCA_002686955.1p__Gemmatimonadota_Ao__GCA-2686955f__GCA-2686955s__GCA-2686955_sp002686955
GCA_002763895.1p__Bacteroidotao__Chlorobialesf__Chloroherpetonaceaes__Thermochlorobacter_sp002763895
GCA_003158365.1p__Bacteroidotao__SJA-28f__B-1ARs__FEN-1279_sp003158365


0

In [14]:
#trim to amplicon region
def trim_aln_to_amp(original_fasta,trim_fasta,start_pos,length):
    with open(original_fasta) as original_fasta:
        with open(trim_fasta, 'w') as trim_fasta:
            records = SeqIO.parse(original_fasta, 'fasta')
            for record in records:
                record.seq = record.seq[start_pos:] #trim at position indicated by cutadapt
                record.seq = [ch for ch in record.seq if ch != '-']
                record.seq = ''.join(record.seq[:length])
                record.seq = Seq.Seq(record.seq)
                SeqIO.write(record, trim_fasta, 'fasta')

#use primer seqs to search aln, and find starting position of trim                
#CGGAGGTAARTTCGAYAAAGG
#GGKFDKG
trim_aln_to_amp(f'{outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.fasta.aln',
                f'{outdir}/gyrb_fastas/gtdbtk_gyrb_Bt_amplicon.fasta',
                707,
                250)
trim_aln_to_amp(f'{outdir}/gyrb_fastas/gtdbtk_gyrb_Bt.faa.aln',
                f'{outdir}/gyrb_fastas/gtdbtk_gyrb_Bt_amplicon.faa',
                236,
                83)


In [15]:
%%bash 
cd results/gyrb/processing/ref_gyrb_gtdbtk
fasttree -gtr -nt < gyrb_fastas/gtdbtk_gyrb_Bt.fasta.aln > gyrb_fastas/gtdbtk_gyrb_Bt.fasta.aln.tre

FastTree Version 2.1.10 Double precision (No SSE3)
Alignment: standard input
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Generalized Time-Reversible, CAT approximation with 20 rate categories
      0.17 seconds: Top hits for    267 of    895 seqs (at seed    100)
      0.32 seconds: Top hits for    456 of    895 seqs (at seed    300)
      0.50 seconds: Top hits for    650 of    895 seqs (at seed    500)
      0.64 seconds: Top hits for    799 of    895 seqs (at seed    700)
      0.74 seconds: Checking top hits for      1 of    895 seqs
      0.92 seconds: Joined    100 of    892
      1.33 seconds: Joined    200 of    892
      1.66 seconds: Joined    300 of    892
      2.17 seconds: Joined    400 of    892
      2.67 seconds: Joined    500 of    892
      3.14 seconds: Joined    600 of    892
      3.66 seconds: Joined    700 of    89